# Carga de datos a PubSub

Este notebook se utilizará como una máquina que está cargando datos de forma continua a PubSub.

# TODO (Datos)

Empecemos cargando los datos

In [5]:
import pandas as pd
from datetime import datetime, timedelta
import json

df = pd.read_csv('TXT_Simulación_TICKETS_2019-12-15.txt', parse_dates=['ticketDate'])

Vamos a introducir las credenciales en esta carpeta:

```
C:\path_to_repository\prevision-dia-tiempo-real\4. Subiendo la App a Heroku\Subir datos a PubSub /
    credentials.json
    Carga de datos a PubSub.ipynb
```

In [6]:
%env GOOGLE_APPLICATION_CREDENTIALS=credentials.json

env: GOOGLE_APPLICATION_CREDENTIALS=credentials.json


In [7]:
from google.cloud import pubsub_v1

# TODO especificar nuestro 'project_id' y 'topic_name'
project_id = ''
topic_name = ''

# Configure the batch to publish as soon as there is one kilobyte
# of data or one second has passed.
batch_settings = pubsub_v1.types.BatchSettings(
    max_bytes=1024,  # One kilobyte
    max_latency=1,   # One second
)
publisher = pubsub_v1.PublisherClient(batch_settings)
topic_path = publisher.topic_path(project_id, topic_name)

Vamos a cargar todos los tickets hasta el minuto actual

In [8]:
def first_load():
    now = datetime.now() + timedelta(days=18)
    now -= timedelta(seconds=now.second)
    now -= timedelta(microseconds=now.microsecond)
    
    df_now = df[df['ticketDate'] <= now]
    for index, row in df_now.iterrows():
        json_values = {}
        json_values['amount'] = row['amount']
        json_values['ticketDate'] = row['ticketDate'].strftime('%Y-%m-%d %H:%M:%S')
        
        # Convertir diccionario en JSON
        json_values = json.dumps(json_values)
        
        # Codificar String
        json_values = json_values.encode('utf-8')
        future = publisher.publish(topic_path, data=json_values)
        
    print('Mensajes Publicados')

para después subirlos minuto a minuto

In [11]:
def run():
    now = datetime.now() + timedelta(days=18)
    now -= timedelta(seconds=now.second)
    now -= timedelta(microseconds=now.microsecond)
    
    df_now = df[df['ticketDate'] == now]
    for index, row in df_now.iterrows():
        json_values = {}
        json_values['amount'] = row['amount']
        json_values['ticketDate'] = row['ticketDate'].strftime('%Y-%m-%d %H:%M:%S')
        
        # Convertir diccionario en JSON
        json_values = json.dumps(json_values)
        
        # Codificar String
        json_values = json_values.encode('utf-8')
        future = publisher.publish(topic_path, data=json_values)
        
    print('Mensajes Publicados')

Para generar un bucle que se repita cada minuto, vamos a utilizar el módulo `APScheduler` de Python

In [12]:
from apscheduler.schedulers.blocking import BlockingScheduler

sched = BlockingScheduler()
sched.add_job(run, 'interval', minutes=1)

<Job (id=6676b63cf967490c9b8abd7c39f45dcb name=run)>

Ejecutemos...

In [ ]:
first_load()
sched.start()

Mensajes Publicados
Mensajes Publicados
b'{"amount": 11.55, "ticketDate": "2019-12-15 14:36:00"}'
Mensajes Publicados
b'{"amount": 18.71, "ticketDate": "2019-12-15 14:37:00"}'


Job "run (trigger: interval[0:01:00], next run at: 2019-11-27 14:39:22 CET)" raised an exception
Traceback (most recent call last):
  File "C:\Users\Asier.Badiola\AppData\Local\Programs\Python\Python37\lib\site-packages\apscheduler\executors\base.py", line 125, in run_job
    retval = job.func(*job.args, **job.kwargs)
  File "<ipython-input-11-0b9f75692eaa>", line 20, in run
    print(json_values)
UnboundLocalError: local variable 'json_values' referenced before assignment


Mensajes Publicados


Job "run (trigger: interval[0:01:00], next run at: 2019-11-27 14:40:22 CET)" raised an exception
Traceback (most recent call last):
  File "C:\Users\Asier.Badiola\AppData\Local\Programs\Python\Python37\lib\site-packages\apscheduler\executors\base.py", line 125, in run_job
    retval = job.func(*job.args, **job.kwargs)
  File "<ipython-input-11-0b9f75692eaa>", line 20, in run
    print(json_values)
UnboundLocalError: local variable 'json_values' referenced before assignment


Mensajes Publicados


...

Mensajes Publicados


Job "run (trigger: interval[0:01:00], next run at: 2019-11-27 14:48:22 CET)" raised an exception
Traceback (most recent call last):
  File "C:\Users\Asier.Badiola\AppData\Local\Programs\Python\Python37\lib\site-packages\apscheduler\executors\base.py", line 125, in run_job
    retval = job.func(*job.args, **job.kwargs)
  File "<ipython-input-11-0b9f75692eaa>", line 20, in run
    print(json_values)
UnboundLocalError: local variable 'json_values' referenced before assignment


Mensajes Publicados


Job "run (trigger: interval[0:01:00], next run at: 2019-11-27 14:49:22 CET)" raised an exception
Traceback (most recent call last):
  File "C:\Users\Asier.Badiola\AppData\Local\Programs\Python\Python37\lib\site-packages\apscheduler\executors\base.py", line 125, in run_job
    retval = job.func(*job.args, **job.kwargs)
  File "<ipython-input-11-0b9f75692eaa>", line 20, in run
    print(json_values)
UnboundLocalError: local variable 'json_values' referenced before assignment


Mensajes Publicados
